In [11]:
##################################### DELETE ALL DATA FROM TABLE ####################
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# --- CONFIGURATION ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

def delete_table_data():
    # 1. Safety Check
    confirm = input(f"⚠️ ARE YOU SURE you want to delete ALL rows from '{DB_TABLE}'? (Type 'yes' to confirm): ")
    if confirm.lower() != "yes":
        print("Action cancelled.")
        return

    try:
        print(f"Connecting to {DB_SERVER}...")
        
        # 2. Encode Password & Connect
        encoded_pass = quote_plus(DB_PASS)
        db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
        engine = create_engine(db_url)

        # 3. Execute Delete
        with engine.connect() as conn:
            # Using 'DELETE' instead of 'TRUNCATE' is safer regarding permissions
            # sql = text(f"DELETE FROM {DB_TABLE} where Filename like 'FLIR0058.jpg'")
            sql = text(f"DELETE FROM {DB_TABLE}")
            result = conn.execute(sql)
            conn.commit()
            print(f"✅ Success! Table '{DB_TABLE}' has been cleared.")
            
    except Exception as e:
        print(f"❌ Error deleting data: {e}")

# Run the delete function
delete_table_data()

Connecting to PSQLAPPEG297-01...
✅ Success! Table 'ThermalReadings' has been cleared.


In [ ]:
import random
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import io
import base64
import matplotlib.pyplot as plt
import numpy as np

# --- CONFIGURATION ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

# Year to generate data for
START_DATE = datetime(2025, 1, 1)
END_DATE = datetime(2025, 12, 31)

# --- ASSET DEFINITIONS ---
# Format: (Section, MachineID, AssetName, AssetCode, Trend_Type)
# Trend Types: 'stable', 'critical_spike', 'steady_warming', 'steady_cooling'
ASSETS = [
    {"code": "TestCodeBuilding-01", "name": "TestName-build", "trend": "stable", "base_temp": 35},
    {"code": "TestCodeCuring-01",   "name": "TestName-curing", "trend": "critical_spike", "base_temp": 50},
    {"code": "TestCodeMixer-01",    "name": "TestName-mixer",  "trend": "steady_warming", "base_temp": 40},
    {"code": "TestCodeSemi-01",     "name": "TestName-semi",   "trend": "steady_cooling", "base_temp": 80},
]

def get_db_engine():
    encoded_pass = quote_plus(DB_PASS)
    db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
    return create_engine(db_url, fast_executemany=True)

def generate_dummy_image(temp_val):
    """Generates a tiny 10x10 heat map image string to prevent broken image icons."""
    data = np.random.rand(10, 10) * temp_val # Random noise scaled by temp
    buffer = io.BytesIO()
    plt.imsave(buffer, data, cmap='inferno', format='jpeg')
    buffer.seek(0)
    return f"data:image/jpeg;base64,{base64.b64encode(buffer.getvalue()).decode('utf-8')}"

def calculate_temp(asset, date_obj):
    """Calculates specific temperature based on the asset's assigned trend."""
    
    # 1. Seasonality (Hotter in July/Aug)
    day_of_year = date_obj.timetuple().tm_yday
    season_factor = 1 + (0.15 * np.sin((day_of_year - 100) / 365 * 2 * np.pi)) # +/- 15% Swing
    
    base = asset['base_temp']
    trend = asset['trend']
    noise = random.uniform(-1.5, 1.5) # Natural variation

    final_temp = base
    
    if trend == 'stable':
        final_temp = (base * season_factor) + noise
        
    elif trend == 'critical_spike':
        # Normal most of the year, but SPIKES huge in July (Days 180-210)
        if 180 < day_of_year < 210:
            final_temp = 75 + random.uniform(0, 5) # Trigger Critical (>70)
        else:
            final_temp = (base * season_factor) + noise

    elif trend == 'steady_warming':
        # Increases by ~2 degrees every month
        month_idx = date_obj.month
        increase = month_idx * 2.0 
        final_temp = base + increase + noise

    elif trend == 'steady_cooling':
        # Decreases by ~2 degrees every month
        month_idx = date_obj.month
        decrease = month_idx * 2.5
        final_temp = base - decrease + noise

    return round(final_temp, 1)

def generate_mock_weather(date_obj):
    """Simple mock weather: Cold in Jan, Hot in Aug."""
    month = date_obj.month
    if month in [12, 1, 2]: return round(random.uniform(10, 15), 1)
    if month in [3, 4, 11]: return round(random.uniform(16, 22), 1)
    if month in [5, 10]:    return round(random.uniform(23, 27), 1)
    return round(random.uniform(28, 35), 1) # Summer

def run_mock_generator():
    print(f"🚀 Generating Mock Data for year {START_DATE.year}...")
    
    engine = get_db_engine()
    rows = []
    
    # Iterate week by week for 1 year
    current_date = START_DATE
    while current_date <= END_DATE:
        
        week_str = current_date.strftime("%Y-%W")
        print(f"   Processing Week: {week_str}...", end='\r')

        for asset in ASSETS:
            # Randomize the exact time (9 AM to 2 PM)
            hour = random.randint(9, 14)
            minute = random.randint(0, 59)
            timestamp = current_date.replace(hour=hour, minute=minute)
            
            # DB Timestamp (UTC, so subtract 2 hours from 'Local' mock time)
            db_timestamp = timestamp - timedelta(hours=2)
            ts_str = db_timestamp.strftime("%Y-%m-%d %H:%M:%S")

            # Calculate Temps
            center_temp = calculate_temp(asset, timestamp)
            max_temp = center_temp + random.uniform(2, 5)
            min_temp = center_temp - random.uniform(2, 5)
            avg_temp = center_temp - random.uniform(0, 1)
            
            row = {
                "Timestamp": ts_str,
                "Filename": f"MOCK_{asset['code']}_{timestamp.strftime('%Y%m%d')}.jpg",
                "Camera_Serial": 999999,
                "Asset_Name": asset['code'], # Linking to 'Asset Code' column
                "Max_Temp_C": round(max_temp, 1),
                "Min_Temp_C": round(min_temp, 1),
                "Center_Temp_C": center_temp,
                "Avg_Temp_C": round(avg_temp, 1),
                "Delta_Temp_C": round(max_temp - min_temp, 1),
                "Emissivity": 0.95,
                "Distance": 2.0,
                "weather_temp": generate_mock_weather(timestamp),
                "Image_Base64": generate_dummy_image(center_temp) 
            }
            rows.append(row)
        
        # Advance by ~7 days (weekly scans)
        current_date += timedelta(days=7)

    print(f"\n📦 Inserting {len(rows)} rows into database...")
    
    df = pd.DataFrame(rows)
    try:
        df.to_sql(DB_TABLE, engine, if_exists='append', index=False)
        print("✅ Success! Database populated.")
    except Exception as e:
        print(f"❌ Error inserting data: {e}")

if __name__ == "__main__":
    run_mock_generator()

Connecting to PSQLAPPEG297-01...
Preparing to insert 20 rows...
✅ Success! Mock data inserted.


In [33]:
import requests
from datetime import datetime

# --- CONFIG ---
ALEX_LAT = 31.2001
ALEX_LON = 29.9187

def get_weather_for_string(time_str):
    """
    Parses a string like "21-Jan-26 11:43 AM" and fetches the weather
    for that specific date and hour.
    """
    try:
        # 1. Parse the string into a datetime object
        # %d=Day, %b=Month(Jan), %y=Year(2-digit), %I=12hr, %M=Min, %p=AM/PM
        dt_obj = datetime.strptime(time_str, "%d-%b-%y %I:%M %p")
        
        # 2. Extract Date and Hour for the API
        date_api_format = dt_obj.strftime("%Y-%m-%d")
        hour_idx = dt_obj.hour  # 11:43 AM becomes index 11
        
        print(f"🔎 Processing: '{time_str}'")
        print(f"   -> Date: {date_api_format} | Hour Index: {hour_idx}")

        # 3. Setup API Request
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": ALEX_LAT,
            "longitude": ALEX_LON,
            "hourly": "temperature_2m",
            "start_date": date_api_format,
            "end_date": date_api_format,
            "timezone": "auto"
        }
        
        response = requests.get(url, params=params, timeout=5)
        
        if response.status_code == 200:
            data = response.json()
            if "hourly" in data and "temperature_2m" in data["hourly"]:
                temps = data["hourly"]["temperature_2m"]
                
                # Check if the hour exists in the returned data
                if 0 <= hour_idx < len(temps):
                    temp = temps[hour_idx]
                    print(f"✅ Temperature at {time_str}: {temp}°C")
                    return temp
                else:
                    print("❌ Hour not found in API response.")
            else:
                print("❌ 'hourly' data missing from response.")
        else:
            print(f"❌ API returned status: {response.status_code}")
            
    except ValueError as e:
        print(f"❌ Date Format Error. Make sure it matches 'DD-Mon-YY HH:MM AM/PM'.\nDetails: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")

if __name__ == "__main__":
    # --- PUT YOUR TIME HERE ---
    target_time = "12-Jan-26 02:47 PM"
    
    get_weather_for_string(target_time)

🔎 Processing: '12-Jan-26 02:47 PM'
   -> Date: 2026-01-12 | Hour Index: 14
✅ Temperature at 12-Jan-26 02:47 PM: 16.9°C


In [26]:
import requests

def get_alexandria_weather(dt_obj):
    """
    Fetches the temperature in Alexandria for the specific hour (No Interpolation).
    """
    try:
        date_str = dt_obj.strftime("%Y-%m-%d")
        hour_idx = dt_obj.hour  # e.g., 11:43 -> 11
        
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": ALEX_LAT,
            "longitude": ALEX_LON,
            "hourly": "temperature_2m",
            "start_date": date_str,
            "end_date": date_str,
            "timezone": "auto"
        }
        
        response = requests.get(url, params=params, timeout=3)

        if response.status_code == 200:
            data = response.json()
            if "hourly" in data and "temperature_2m" in data["hourly"]:
                temps = data["hourly"]["temperature_2m"]
                
                # Directly grab the temp for this hour index
                if 0 <= hour_idx < len(temps):
                    return float(temps[hour_idx])
                    
        return None
    except Exception as e:
        logging.error(f"⚠️ Weather API Error: {e}")
        return None
    
print(get_alexandria_weather(datetime.strptime("18-Jan-26 9:54 AM", "%d-%b-%y %I:%M %p")))


17.3


In [28]:
import requests
import time
from datetime import datetime
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# --- CONFIG ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

# --- TIME RANGE TO FIX (Inclusive) ---
FIX_START_DATE = "2026-01-12 00:00:00"
FIX_END_DATE   = "2026-01-21 23:59:59"

# Location
ALEX_LAT = 31.2001
ALEX_LON = 29.9187

# Cache to avoid spamming the API (Key: "YYYY-MM-DD-HH", Value: Temp)
weather_cache = {}

def get_alexandria_weather_cached(dt_obj):
    """
    Fetches weather for the specific hour.
    Checks cache first to speed up processing.
    """
    # Create a unique key for this hour (e.g., "2026-01-21-09")
    cache_key = dt_obj.strftime("%Y-%m-%d-%H")
    
    if cache_key in weather_cache:
        return weather_cache[cache_key]

    # --- API CALL ---
    try:
        date_str = dt_obj.strftime("%Y-%m-%d")
        hour_idx = dt_obj.hour
        
        print(f"   🌍 Fetching API data for: {date_str} Hour: {hour_idx}...")
        
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            "latitude": ALEX_LAT,
            "longitude": ALEX_LON,
            "hourly": "temperature_2m",
            "start_date": date_str,
            "end_date": date_str,
            "timezone": "auto"
        }
        
        response = requests.get(url, params=params, timeout=5)

        if response.status_code == 200:
            data = response.json()
            if "hourly" in data and "temperature_2m" in data["hourly"]:
                temps = data["hourly"]["temperature_2m"]
                if 0 <= hour_idx < len(temps):
                    temp = float(temps[hour_idx])
                    # Save to cache
                    weather_cache[cache_key] = temp
                    return temp
        
        print(f"   ❌ Weather data unavailable for {cache_key}")
        return None

    except Exception as e:
        print(f"   ❌ API Error: {e}")
        return None

def run_weather_fix():
    print(f"🚀 Starting Weather Fix from {FIX_START_DATE} to {FIX_END_DATE}...")
    
    encoded_pass = quote_plus(DB_PASS)
    db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(db_url)

    try:
        with engine.connect() as conn:
            # 1. Get the list of files to update
            select_query = text(f"""
                SELECT Filename, Timestamp 
                FROM {DB_TABLE} 
                WHERE Timestamp >= :start AND Timestamp <= :end
            """)
            
            result = conn.execute(select_query, {"start": FIX_START_DATE, "end": FIX_END_DATE})
            rows = result.fetchall()
            
            if not rows:
                print("✅ No records found in this range.")
                return

            print(f"📋 Found {len(rows)} records. Processing...")

            updated_count = 0
            
            # 2. Iterate and Update
            for row in rows:
                filename = row[0]
                timestamp = row[1] # This is already a datetime object from SQL

                # Get weather (Cached version)
                new_temp = get_alexandria_weather_cached(timestamp)

                if new_temp is not None:
                    update_query = text(f"""
                        UPDATE {DB_TABLE} 
                        SET weather_temp = :temp 
                        WHERE Filename = :fname
                    """)
                    conn.execute(update_query, {"temp": new_temp, "fname": filename})
                    updated_count += 1
            
            conn.commit()
            print(f"🎉 Success! Updated {updated_count} records.")

    except Exception as e:
        print(f"❌ Database Error: {e}")

if __name__ == "__main__":
    run_weather_fix()

🚀 Starting Weather Fix from 2026-01-12 00:00:00 to 2026-01-21 23:59:59...
📋 Found 65 records. Processing...
   🌍 Fetching API data for: 2026-01-12 Hour: 14...
   🌍 Fetching API data for: 2026-01-12 Hour: 15...
   🌍 Fetching API data for: 2026-01-18 Hour: 9...
   🌍 Fetching API data for: 2026-01-20 Hour: 11...
   🌍 Fetching API data for: 2026-01-21 Hour: 9...
   🌍 Fetching API data for: 2026-01-18 Hour: 10...
🎉 Success! Updated 65 records.


In [35]:
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

# --- CONFIG ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

# --- DATE RANGE TO DELETE (Inclusive) ---
DELETE_FROM_DATE = "2025-01-1 00:00:00"
DELETE_END_DATE  = "2026-01-21 23:59:59"

def delete_range_records():
    # 1. Setup Connection
    encoded_pass = quote_plus(DB_PASS)
    db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
    engine = create_engine(db_url)

    try:
        with engine.connect() as conn:
            # 2. Check how many records match the range
            count_query = text(f"""
                SELECT COUNT(*) FROM {DB_TABLE} 
                WHERE Timestamp >= :start_date AND Timestamp <= :end_date
            """)
            
            params = {"start_date": DELETE_FROM_DATE, "end_date": DELETE_END_DATE}
            result = conn.execute(count_query, params).scalar()

            if result == 0:
                print(f"✅ No records found between {DELETE_FROM_DATE} and {DELETE_END_DATE}.")
                return

            print(f"⚠️  WARNING: You are about to DELETE {result} records.")
            print(f"   📅 Range: {DELETE_FROM_DATE}  -->  {DELETE_END_DATE}")
            confirm = input("Type 'DELETE' to confirm: ")

            if confirm == "DELETE":
                # 3. Perform Deletion
                delete_query = text(f"""
                    DELETE FROM {DB_TABLE} 
                    WHERE Timestamp >= :start_date AND Timestamp <= :end_date
                """)
                conn.execute(delete_query, params)
                conn.commit()
                print(f"🗑️  Success: {result} records deleted.")
            else:
                print("❌ Operation cancelled.")

    except Exception as e:
        print(f"❌ Database Error: {e}")

if __name__ == "__main__":
    delete_range_records()

⚠️  WARNING: You are about to DELETE 4 records.
   📅 Range: 2025-01-1 00:00:00  -->  2026-01-21 23:59:59
🗑️  Success: 4 records deleted.


In [19]:
import pyodbc

# --- CONFIGURATION ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

CONN_STR = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={DB_SERVER};"
    f"DATABASE={DB_NAME};"
    f"UID={DB_USER};"
    f"PWD={DB_PASS}"
)

def delete_mock_data():
    try:
        conn = pyodbc.connect(CONN_STR)
        cursor = conn.cursor()
        sql = f"DELETE FROM {DB_TABLE} WHERE Camera_Serial = 9999"
        print("Cleaning up existing mock rows...")
        cursor.execute(sql)
        conn.commit()
        print(f"Deleted {cursor.rowcount} rows.")
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    delete_mock_data()

Cleaning up existing mock rows...
Deleted 0 rows.


In [4]:
import os
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from dotenv import load_dotenv

# --- LOAD CONFIGURATION ---
load_dotenv()

DB_SERVER = os.getenv("DB_SERVER")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")

# --- THE CLEANUP QUERY ---
# We use simple table names here because the connection is already 
# inside the correct database.
cleanup_query = """
WITH CTE AS (
    SELECT 
        [Timestamp], 
        [Asset_Name], 
        [Camera_Serial], 
        ROW_NUMBER() OVER (
            PARTITION BY [Timestamp], [Asset_Name], [Camera_Serial] 
            ORDER BY [Timestamp] DESC
        ) AS RowNum
    FROM [ThermalReadings]
)
DELETE FROM CTE WHERE RowNum > 1;
"""

def clean_database():
    print("-------------------------------------------------------")
    print("🧹 Starting Database Cleanup...")

    if not all([DB_SERVER, DB_NAME, DB_USER, DB_PASS]):
        print("❌ Error: Missing database credentials in .env file.")
        return

    try:
        # 1. Connect to Database
        encoded_pass = quote_plus(DB_PASS)
        db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
        engine = create_engine(db_url)

        # 2. Execute the Delete Query
        with engine.connect() as conn:
            print(f"🔌 Connected to {DB_NAME}. Running cleanup query on [ThermalReadings]...")
            
            # SQLAlchemy requires .commit() for DELETE/UPDATE operations
            result = conn.execute(text(cleanup_query))
            conn.commit()
            
            print(f"✅ Success! Rows affected: {result.rowcount}")

    except Exception as e:
        print(f"❌ Error during cleanup: {e}")
        
        # Fallback: If [ThermalReadings] failed, try [Flir].[ThermalReadings]
        if "Invalid object name" in str(e):
            print("\n⚠️ Retrying with 'Flir' schema...")
            try:
                cleanup_query_flir = cleanup_query.replace("[ThermalReadings]", "[Flir].[ThermalReadings]")
                with engine.connect() as conn:
                    result = conn.execute(text(cleanup_query_flir))
                    conn.commit()
                    print(f"✅ Success with Flir schema! Rows affected: {result.rowcount}")
            except Exception as e2:
                print(f"❌ Failed again: {e2}")

    finally:
        if 'engine' in locals():
            engine.dispose()
        print("-------------------------------------------------------")

if __name__ == "__main__":
    clean_database()

-------------------------------------------------------
🧹 Starting Database Cleanup...
🔌 Connected to Flir. Running cleanup query on [ThermalReadings]...
✅ Success! Rows affected: 29
-------------------------------------------------------
